In [104]:
import sqlite3
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
# pip install "gluonts[torch]"

# Connect to the SQLite database for training data
train_conn = sqlite3.connect('MegaMillions_Train.db')

# Query data from the database for training
query_train = "SELECT * FROM Interval_Data"
df_train = pd.read_sql_query(query_train, train_conn)
df_train

# Connect to the SQLite database for test data
test_conn = sqlite3.connect('MegaMillions_Test.db')

# Query data from the database for test
query_test = "SELECT * FROM Interval_Data"
df_test = pd.read_sql_query(query_test, test_conn)
df_test

,Draw Date,Winning Numbers,Mega Ball,Multiplier
0,10/19/2018,15 23 53 65 70,7,2
1,10/19/2018,15 23 53 65 70,7,2
2,10/19/2018,15 23 53 65 70,7,2
3,10/19/2021,03 12 13 19 52,1,3
4,10/19/2021,03 12 13 19 52,1,3
...,...,...,...,...
628,12/31/2019,30 44 49 53 56,11,3
629,12/31/2019,30 44 49 53 56,11,3
630,12/31/2021,02 05 30 46 61,8,3
631,12/31/2021,02 05 30 46 61,8,3


In [105]:
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.evaluation import make_evaluation_predictions, Evaluator
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# pip install "gluonts[torch]"

# Convert 'Draw Date' to datetime
df_train['Draw Date'] = pd.to_datetime(df_train['Draw Date'])

# Prepare DataFrame for predicting Winning Numbers
winning_numbers = df_train['Winning Numbers'].str.split(' ', expand=True)
winning_numbers.columns = [f'Ball_{i+1}' for i in winning_numbers.columns]
df_train = pd.concat([df_train, winning_numbers], axis=1)

# Prepare data for predicting Winning Numbers and Mega Ball
winning_numbers_df = df_train[['Draw Date'] + [f'Ball_{i+1}' for i in range(5)] + ['Mega Ball']].copy()
winning_numbers_df.columns = ['ds'] + [f'y_{i+1}' for i in range(5)] + ['y_mega_ball']  # Rename columns

# Min-Max scaling for the numerical columns
scaler = MinMaxScaler()
numerical_columns = [f'y_{i+1}' for i in range(5)] + ['y_mega_ball']
winning_numbers_df[numerical_columns] = scaler.fit_transform(winning_numbers_df[numerical_columns])

# Create a ListDataset for multiple values prediction using the scaled data
data = [
    {
        "start": winning_numbers_df.iloc[i]['ds'],
        "target": winning_numbers_df.iloc[i][numerical_columns].values,
        "feat_dynamic_real": [[None] * 6]  
    }
    for i in range(len(winning_numbers_df))
]

# Frequency of data 
freq = "D"

# Create ListDataset
train_ds = ListDataset(data, freq=freq)

# Define DeepAR estimator for predicting 5 white balls and 1 Mega Ball
estimator = DeepAREstimator(prediction_length=6, freq=freq, context_length=30, num_layers=3)

# Train the model
predictor = estimator.train(training_data=train_ds)

# Make predictions for Winning Numbers and Mega Ball
forecast_it, ts_it = make_evaluation_predictions(dataset=train_ds, predictor=predictor, num_samples=10)

forecasts = list(forecast_it)
tss = list(ts_it)

# Calculate metrics
evaluator = Evaluator()
agg_metrics, item_metrics = evaluator(tss, forecasts, num_series=len(train_ds))

mae_white_balls = agg_metrics["MASE"]
mse_white_balls = agg_metrics["MSE"]
rmse_white_balls = agg_metrics["RMSE"]

print(f"MAE for White Balls and Mega Ball: {mae_white_balls}")
print(f"MSE for White Balls and Mega Ball: {mse_white_balls}")
print(f"RMSE for White Balls and Mega Ball: {rmse_white_balls}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name  | Type        | Params | In sizes                                                        | Out sizes  
----------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 39.0 K | [[1, 1], [1, 1], [1, 1122, 4], [1, 1122], [1, 1122], [1, 6, 4]] | [1, 100, 6]
----------------------------------------------------------------------------------------------------------------------
39.0 K    Trainable params
0         Non-trainable params
39.0 K    Total params
0.156     Total estimated model params size (MB)


Training: |                                                                                                   …

Epoch 0, global step 50: 'train_loss' reached 8.17453 (best 8.17453), saving model to 'C:\\Users\\Ethan\\PycharmProjects\\MegaMillionsProject\\lightning_logs\\version_19\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 6.76985 (best 6.76985), saving model to 'C:\\Users\\Ethan\\PycharmProjects\\MegaMillionsProject\\lightning_logs\\version_19\\checkpoints\\epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 6.60185 (best 6.60185), saving model to 'C:\\Users\\Ethan\\PycharmProjects\\MegaMillionsProject\\lightning_logs\\version_19\\checkpoints\\epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 6.54581 (best 6.54581), saving model to 'C:\\Users\\Ethan\\PycharmProjects\\MegaMillionsProject\\lightning_logs\\version_19\\checkpoints\\epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 6.46320 (best 6.46320), saving model to 'C:\\Users\\Ethan\\PycharmProjects\\MegaMillion

MAE for White Balls and Mega Ball: nan
MSE for White Balls and Mega Ball: 0.2998087633068742
RMSE for White Balls and Mega Ball: 0.5475479552576872



C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\pandas\core\dtypes\astype.py:134: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\pandas\core\dtypes\astype.py:134: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\pandas\core\dtypes\astype.py:134: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [64]:
'''# Convert 'Draw Date' to datetime
df_test['Draw Date'] = pd.to_datetime(df_test['Draw Date'])

# Split the 'Winning Numbers' into separate columns for white balls
winning_numbers_test = df_test['Winning Numbers'].str.split(' ', expand=True)
winning_numbers_test.columns = [f'Ball_{i+1}' for i in winning_numbers_test.columns]
df_test = pd.concat([df_test, winning_numbers_test], axis=1)

# Convert 'Ball' columns to integers
ball_columns_test = [f'Ball_{i+1}' for i in range(5)]  # Assuming 5 white balls
for col in ball_columns_test:
    df_test[col] = pd.to_numeric(df_test[col])

# Prepare data for predicting Winning Numbers and Mega Ball on test data
test_data = df_test[['Draw Date'] + ball_columns_test + ['Mega Ball']].copy()
test_data.columns = ['ds'] + [f'y_{i+1}' for i in range(5)] + ['y_mega_ball']  # Rename columns

# Create a ListDataset for the test data including white balls and Mega Ball
test_data_list = [
    {
        "start": test_data.iloc[i]['ds'],
        "target": test_data.iloc[i][[f'y_{i+1}' for i in range(5)] + ['y_mega_ball']].values,
        "feat_dynamic_real": [[None] * 6]  # 2D array of Nones for features (if any)
    }
    for i in range(len(test_data))
]

# Frequency of data (assuming daily frequency)
freq = "D"

# Create ListDataset for the test data
test_ds = ListDataset(test_data_list, freq=freq)

# Use the trained predictor to make predictions on the test data
forecast_it_test, ts_it_test = make_evaluation_predictions(
    dataset=test_ds,
    predictor=predictor,
    num_samples=100  # Number of sample paths to draw while making predictions
)

forecasts_test = list(forecast_it_test)
tss_test = list(ts_it_test)
# Calculate metrics or perform any analysis needed for evaluation
evaluator = Evaluator()
agg_metrics_test, item_metrics_test = evaluator(iter(tss_test), iter(forecasts_test), num_series=len(test_ds))

# Retrieve metrics for white balls and Mega Ball from the test data evaluation
mae_white_balls_test = agg_metrics_test["MASE"]
mse_white_balls_test = agg_metrics_test["MSE"]
rmse_white_balls_test = agg_metrics_test["RMSE"]

print(f"MAE for White Balls on test data: {mae_white_balls_test}")
print(f"MSE for White Balls on test data: {mse_white_balls_test}")
print(f"RMSE for White Balls on test data: {rmse_white_balls_test}")
'''

Running evaluation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 633/633 [00:03<00:00, 169.75it/s]

MAE for White Balls on test data: nan
MSE for White Balls on test data: 1354.691943127962
RMSE for White Balls on test data: 36.806140019403855



C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\pandas\core\dtypes\astype.py:134: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\pandas\core\dtypes\astype.py:134: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
C:\Users\Ethan\PycharmProjects\pythonProject6\venv\lib\site-packages\pandas\core\dtypes\astype.py:134: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [103]:
def predict_mega_ball(model, date):
    # Prepare data for prediction based on the provided date
    prediction_data = pd.DataFrame({'ds': [date], 'y': [0]})  # Replace 0 with a default value for prediction

    # Convert 'ds' column to datetime
    prediction_data['ds'] = pd.to_datetime(prediction_data['ds'])

    # Create a ListDataset for prediction
    prediction_ds = ListDataset(
        [
            {
                "start": prediction_data["ds"].values[0],  # Assuming a single time series
                "target": prediction_data["y"].values,
                "feat_dynamic_real": [[None]]  # 2D array of a single None
            }
        ],
        freq="D"
    )

    # Make prediction using the provided model
    forecast_it_pred = model.predict(prediction_ds)

    # Extract the predicted value for the Mega Ball
    forecast = next(forecast_it_pred)
    predicted_mega_ball = forecast.mean[0]

    return predicted_mega_ball



predicted_number = predict_mega_ball(predictor, '2023-12-31')  
print(f"Predicted Mega Ball number for 2023-12-31: {predicted_number}")


Predicted Mega Ball number for 2023-12-31: -1.3140626087260188e-10


In [98]:


# Frequency of data 
freq = "D"

# Create a ListDataset for the input date prediction
def predict_numbers(input_date):
    prediction_data = [
        {
            "start": input_date,
            "target": [None, None, None, None, None, None],  # Placeholder for predictions
            "feat_dynamic_real": [[None] * 6]  
        }
    ]

    predict_ds = ListDataset(prediction_data, freq=freq)

    # Make predictions for the input date
    forecast_it, ts_it = make_evaluation_predictions(dataset=predict_ds, predictor=predictor, num_samples=10)

    forecasts = list(forecast_it)
    tss = list(ts_it)

    # Return the predicted values for white balls and Mega Ball
    return forecasts[0].samples.mean(axis=0)[:5], forecasts[0].samples.mean(axis=0)[-1]

input_date = pd.to_datetime('2023-12-1')
white_balls_pred, mega_ball_pred = predict_numbers(input_date)

# Assuming white_balls_pred and mega_ball_pred are the predicted values
combined_predictions = np.concatenate((white_balls_pred, np.array([mega_ball_pred])))

# Reshape the combined predictions array to fit the scaler
combined_predictions = combined_predictions.reshape(1, -1)


# Fit the scaler on the combined predictions
scaler.fit(combined_predictions)

# Scale the combined predictions
scaled_combined_predictions = combined_scaler.transform(combined_predictions)

# Now, you have the combined and scaled predictions
print(f"Scaled Combined Predictions: {scaled_combined_predictions}")


Scaled Combined Predictions: [[ 1.7714401e-09  6.7203447e-09 -5.6036285e-08 -1.9172457e-08
   1.2283897e-08 -2.0033717e-08]]
